In [1]:
using Revise, WVZAnalysis, UnROOT
# Arrow, DataFrames, MLJ, Plots, ComputationalResources, MLDataUtils
# import StatsBase

┌ Info: Precompiling WVZAnalysis [15e846ca-95d3-4e21-8d51-3cc2ce27e5cd]
└ @ Base loading.jl:1423
┌ Warning: Package WVZAnalysis does not have LinearAlgebra in its dependencies:
│ - If you have WVZAnalysis checked out for development and have
│   added LinearAlgebra as a dependency but haven't updated your primary
│   environment's manifest file, try `Pkg.resolve()`.
│ - Otherwise you may need to report an issue with WVZAnalysis
└ Loading LinearAlgebra into WVZAnalysis from project dependency, future warnings for WVZAnalysis are suppressed.


In [2]:
V13BASE = "/hdd01/minitree/v1.3/"
VVVfiles = joinpath.(V13BASE, readlines(joinpath(@__DIR__, "VVVfile.list")));

In [3]:
BGfiles = joinpath.(V13BASE, readlines(joinpath(@__DIR__, "BGfile.list")));
sort!(BGfiles, by=filesize; rev=true);

In [4]:
thread_hist(files) = reduce((.+), [WVZAnalysis.main_looper(x) for x in files]);
#thread_data(files) = mapreduce(fetch, (x,y) -> append!.(x,y), [Threads.@spawn(WVZAnalysis.main_looper(x)) for x in files]);

In [10]:
@time thread_hist(BGfiles)

 48.217984 seconds (808.56 M allocations: 239.735 GiB, 35.50% gc time, 1.22% compilation time)


3-element Dictionaries.Dictionary{Symbol, FHist.Hist1D{Float32, Tuple{StepRange{Int64, Int64}}}}
 :Z_mass_first │                   ┌                              ┐ 
   …
  :WZZ_ZZ_mass │ 
edges: 0:10:800
bin counts: Float32[0.0, 0.0, 0.0, 0.0, 0.0, 0.…
      :WWZ_MET │ 
edges: 0:5:400
bin counts: Float32[211.15694, 447.93295, 424.18…

In [6]:
WVZAnalysis.main_looper(BGfiles[end])

3-element Dictionaries.Dictionary{Symbol, FHist.Hist1D{Float32, Tuple{StepRange{Int64, Int64}}}}
 :Z_mass_first │                   ┌                              ┐ 
   …
  :WZZ_ZZ_mass │ 
edges: 0:10:800
bin counts: Float32[0.0, 0.0, 0.0, 0.0, 0.0, 0.…
      :WWZ_MET │ 
edges: 0:5:400
bin counts: Float32[0.0, 0.0, 0.0, 0.0, 0.0, 0.0…

In [18]:
SIGhist = @time thread_hist(BGfiles);
# WVZAnalysis.main_looper(VVVfiles[1])

 41.966513 seconds (747.07 M allocations: 214.925 GiB, 27.02% gc time, 0.07% compilation time)


In [23]:
SIGhist[:WWZ_MET]

edges: 0:5:400
bin counts: Float32[211.15753, 447.93393, 424.18665, 349.1948, 292.13525, 217.52031, 157.93633, 115.89529, 87.68743, 58.627426  …  0.00044505246, 0.018249791, 0.0040290384, -0.0011624552, 0.007844416, 0.0040369085, 0.0041590766, 0.0022946307, 0.0055260295, 0.007087419]
total count: 2519.1445

In [19]:
sum(filesize, BGfiles) / 1024^3

53.23512900713831

In [ ]:
# SIGdata = thread_data(BGfiles);
# Arrow.write("/home/jiling/minitree/BG.arrow", Dict(pairs(SIGdata)))

In [ ]:
df_sig = DataFrame(Arrow.Table("/home/jiling/minitree/SIG.arrow"));
df_bg = DataFrame(Arrow.Table("/home/jiling/minitree/BG.arrow"));

df_sig.tag .= 1
df_bg.tag .= 0;

df_all = vcat(df_bg, df_sig);

In [ ]:
df_balance = DataFrame(undersample(x->x.tag, df_all); copycols=true);

In [ ]:
for n in names(df_balance[!, Not([:tag, :wgt])])
    if eltype(df_balance[!, n]) <: Integer
        df_balance[!, n] = float.(df_balance[!, n])
    end
    StatsBase.transform!(StatsBase.fit(StatsBase.UnitRangeTransform, df_balance[!, n]), df_balance[!, n])
end

In [ ]:
# Arrow.write("/home/jiling/minitree/TAG_BALANCE.arrow", df_balance)

In [ ]:
y, X = unpack(df_balance, ==(:tag), x-> x ∉(:tag, :wgt); :tag=>OrderedFactor);

In [ ]:
train, test = partition(eachindex(y), 0.8, shuffle=true);

In [ ]:
# filter(x->true, models(matching(X, y))) .|> println;

In [ ]:
my_pred(machine, xs) = map(x->x.prob_given_ref[2], MLJ.predict(machine, xs))

In [ ]:
function density_plot(machine)
    train_sig = findall(==(1), y[train])
    train_bkg = findall(==(0), y[train])
    BINS = 0:0.02:1
    stephist(my_pred(machine, X[train, :][train_sig, :]), bins=BINS, normalize=true, label="train SIG")
    stephist!(my_pred(machine, X[train, :][train_bkg, :]), bins=BINS,normalize=true,label="train BKG")

    test_sig = findall(==(1), y[test])
    test_bkg = findall(==(0), y[test])
    stephist!(my_pred(machine, X[test, :][test_sig, :]), bins=BINS,normalize=true,label="test SIG")
    stephist!(my_pred(machine, X[test, :][test_bkg, :]), bins=BINS, normalize=true, xlabel="BDT prob.",
        label="test BKG", title="Density plot", legend=:top)
end

## XGBoost

In [ ]:
#CLS = @load AdaBoostStumpClassifier
CLS = @load XGBoostClassifier
nnc = CLS(num_round=100, eta=0.45, max_depth=4);

In [ ]:
# r1 = range(nnc, :eta, lower=0.2, upper=0.5);
# r2 = range(nnc, :max_depth, lower=4, upper=7);
# self_tuning_tree = TunedModel(model=nnc,
#                               operation=MLJ.predict_mode,
#                               resampling=Holdout(fraction_train=0.9, rng=1234),
#                               tuning=Grid(resolution=8),
#                               range=[r1, r2],
#                               measure=misclassification_rate);

In [ ]:
mach = machine(nnc, X, y)

In [ ]:
fit!(mach; rows=train);
# yhat = MLJ.predict(mach, X[train,:])
# mean(cross_entropy(yhat, y[train]))

In [ ]:
# fitted_params(mach).best_model

In [ ]:
yhat = MLJ.predict(mach, X)
fprs, tprs, _ = roc_curve(MLJ.predict(mach, X[test,:]), y[test]);
fprs_t, tprs_t, _ = roc_curve(yhat, y);
plot(fprs, tprs; label="test")
plot!(fprs_t, tprs_t; label="train", xlabel="false positive rate", ylabel="truth positive rate", fmt=:png)

In [ ]:
density_plot(mach)

In [ ]:
import XGBoost

In [ ]:
XGBoost.importance(mach.fitresult[1])

In [ ]:
names(X)[[5,12,1,6,23,3,7,2,18]]

## Option 2

In [ ]:
import Flux
CLS2 = @load NeuralNetworkClassifier
nnc2 = CLS2(optimiser=Flux.Optimise.ADAGrad(), epochs = 100, batch_size=50, acceleration=CUDALibs())

In [ ]:
MLJFlux.build(nnc2, Random.GLOBAL_RNG, (25, 2))

In [ ]:
# iterated_model = IteratedModel(model=nnc2,
#                                measures=cross_entropy,
#                                resampling=Holdout(fraction_train=0.8),
#                                controls=[Step(4),
#                                          Patience(3),
#                                          NumberLimit(100)],
#                                 retrain=true);

In [ ]:
mach2 = machine(nnc2, X, y)
fit!(mach2; rows=train)

In [ ]:
yhat = MLJ.predict(mach2, X)
fprs, tprs, _ = roc(MLJ.predict(mach2, X[test,:]), y[test]);
fprs_t, tprs_t, _ = roc(yhat, y);
plot(fprs, tprs; label="test")
plot!(fprs_t, tprs_t; label="train", xlabel="false positive rate", ylabel="truth positive rate", fmt=:png)

In [ ]:
density_plot(mach2)

## Option 3

In [ ]:
CLS3 = @load EvoTreeClassifier
nnc3 = CLS3(nrounds = 50,  η = 0.1, max_depth=5, device="gpu")

In [ ]:
mach3 = machine(nnc3, X, y)
fit!(mach3, rows=train)

In [ ]:
yhat = MLJ.predict(mach3, X)
fprs, tprs, _ = roc(MLJ.predict(mach3, X[test,:]), y[test]);
fprs_t, tprs_t, _ = roc(yhat, y);
plot(fprs, tprs; label="test")
plot!(fprs_t, tprs_t; label="train", xlabel="false positive rate", ylabel="truth positive rate", fmt=:png)

In [ ]:
density_plot(mach3)